In [1]:
import transformers, torch, datasets
print("transformers", transformers.__version__)
print("torch", torch.__version__)
print("datasets", datasets.__version__)

C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


transformers 4.22.2
torch 1.12.1+cu116
datasets 2.6.1


Подгружаем готовый датасет sst2 из HuggingFace Datasets Hub 

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

dataset = load_dataset("sst2")

Found cached dataset sst2 (C:/Users/Gigabyte/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)
100%|██████████| 3/3 [00:00<00:00, 532.88it/s]


tokenizer - токенизатор для предобработки данных. Токенизируем набор данных - разбиваем его на компоненты понятного для модели вида

In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Loading cached processed dataset at C:/Users/Gigabyte/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5\cache-95ce16d58ec006b6.arrow
Loading cached processed dataset at C:/Users/Gigabyte/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5\cache-67610756c5f04d3f.arrow
Loading cached processed dataset at C:/Users/Gigabyte/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5\cache-06aed57ca092479f.arrow


Загрузим предобученную модель. В переменной model будет предобученная модель distilBERT. Указываем устройство обучения (Cuda, если есть видеокарта Nvidia, иначе ЦП)

In [4]:
from torch import device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.to(device)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [6]:
import evaluate
metric = evaluate.load("accuracy")

In [7]:
metric

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [8]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
validation_key = "validation"

Задаем параметры обучения

In [10]:
training_args = TrainingArguments(
    output_dir="C:\\Users\\Gigabyte\\Desktop\\IT\\oop\\lab1\\results\\",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset[validation_key],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Запускаем обучение сети

In [12]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 67349
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 6315
  8%|▊         | 500/6315 [00:52<09:52,  9.82it/s]Saving model checkpoint to C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\check

{'loss': 0.3341, 'learning_rate': 9.208234362628663e-06, 'epoch': 0.24}


Model weights saved in C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-500\pytorch_model.bin
tokenizer config file saved in C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-500\tokenizer_config.json
Special tokens file saved in C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-500\special_tokens_map.json
 16%|█▌        | 1000/6315 [01:46<09:00,  9.83it/s]Saving model checkpoint to C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-1000
Configuration saved in C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-1000\config.json


{'loss': 0.2492, 'learning_rate': 8.416468725257323e-06, 'epoch': 0.48}


Model weights saved in C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-1000\tokenizer_config.json
Special tokens file saved in C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-1000\special_tokens_map.json
 24%|██▍       | 1500/6315 [02:39<08:03,  9.95it/s]Saving model checkpoint to C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-1500
Configuration saved in C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-1500\config.json


...

Model weights saved in C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-5500\pytorch_model.bin
tokenizer config file saved in C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-5500\tokenizer_config.json
Special tokens file saved in C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-5500\special_tokens_map.json
 95%|█████████▌| 6000/6315 [10:41<00:34,  9.18it/s]Saving model checkpoint to C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-6000
Configuration saved in C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-6000\config.json


{'loss': 0.1092, 'learning_rate': 4.988123515439431e-07, 'epoch': 2.85}


Model weights saved in C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-6000\pytorch_model.bin
tokenizer config file saved in C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-6000\tokenizer_config.json
Special tokens file saved in C:\Users\Gigabyte\Desktop\IT\oop\lab1\results\checkpoint-6000\special_tokens_map.json
100%|██████████| 6315/6315 [11:15<00:00,  9.95it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 6315/6315 [11:15<00:00,  9.34it/s]

{'train_runtime': 675.8833, 'train_samples_per_second': 298.938, 'train_steps_per_second': 9.343, 'train_loss': 0.16769504139372401, 'epoch': 3.0}


TrainOutput(global_step=6315, training_loss=0.16769504139372401, metrics={'train_runtime': 675.8833, 'train_samples_per_second': 298.938, 'train_steps_per_second': 9.343, 'train_loss': 0.16769504139372401, 'epoch': 3.0})

Расчитаем метрики

In [13]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 32
100%|██████████| 28/28 [00:00<00:00, 30.43it/s]


{'eval_loss': 0.33868709206581116,
 'eval_accuracy': 0.9048165137614679,
 'eval_runtime': 0.9709,
 'eval_samples_per_second': 898.093,
 'eval_steps_per_second': 28.838,
 'epoch': 3.0}